In [ ]:
import pandas as pd
import toml
import pygsheets
import re

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api as api
import paulssonlab.cloning.workflow as workflow
import paulssonlab.cloning.registry as registry
import paulssonlab.api.labops as labops

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
reg = registry.Registry(gc, config["registry"]["folder"])

In [ ]:
strains_sheet = gc.open_by_key(reg.registry[("LIB", "strains")]).worksheet()
plasmids_sheet = gc.open_by_key(reg.registry[("pLIB", "plasmids")]).worksheet()
plasmid_maps_folder = reg.registry[("pLIB", "maps")]

# Lab Obs ordering site

In [ ]:
browser = await labops.login_harvard(
    config["ecommons"]["username"],
    config["ecommons"]["password"],
    service="https://sysbiolabops.hms.harvard.edu/casservice",
)

In [ ]:
addgene_orders = await labops.get_orders(browser, vendor="Addgene (26)")

In [ ]:
my_addgene = addgene_orders[
    addgene_orders["Name"].isin(["Jacob Shenker", "Noah Olsman"])
]

In [ ]:
my_addgene.info()

In [ ]:
to_add = my_addgene[my_addgene["Submitted"] > pd.to_datetime("10/1/20")]

In [ ]:
to_add["Catalog #"]

In [ ]:
addgene_catalog = (
    to_add["Catalog #"].apply(lambda x: int(re.search(r"(\d+)$", x).group(1))).values
)

In [ ]:
addgene_data = workflow.import_addgene(
    addgene_catalog, strains_sheet, plasmids_sheet, plasmid_maps_folder
)